In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.messages import HumanMessage
from langchain_core.prompts import *
from langchain.tools import tool
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

@tool
def trident_cve_tool(cve_id: str):
    """
    This is a CVE tool to call trident CVE information.
    It should be used to fetch general information of a specific cve (format like CVE-XXX-XXX).
    The general information from Trident API includes:
    cve id, name, description, update date, publish date and some other public available information.
    """
    return str("Trident CVE data")

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["cve_id"],
                template="You are a helpful assistant to explain a certain CVE {cve_id} in detail",
            )
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["question"],
                template="Please answer this question: {question}",
            )
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
tools = [trident_cve_tool]

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def create_llm():
    return AzureChatOpenAI(
        temperature=0.0,
        azure_endpoint="http://localhost:8000",
        openai_api_key="bedrock",
        openai_api_version="2024-06-01",
        deployment_name="anthropic.claude-3-5-sonnet-20240620-v1:0",    
        # deployment_name="gpt-4o_2024-05-13",
        openai_api_type="azure",
    )




In [ ]:
llm = create_llm()
llm.temperature = 0.0
agent = create_tool_calling_agent(llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
content = agent_executor.invoke(
    {"question": "What is CVE-2020-17087?", "cve_id": "CVE-2020-17087"}
)
print(content)


In [ ]:
# Load a picture from a local file and convert to base64
# from llm_utils import LLMUtils
import base64

# llm = LLMUtils(model_name="anthropic.claude-3-5-sonnet-20240620-v1:0")


content = llm.invoke(
    "What is happening in this image?",
    images=[encoded_string],
    config={"max_tokens": 2, "temperature": 0.5},
)

print(content)

content = llm.invoke(
    "What is the capital of France?",
    config={"max_tokens": 2, "temperature": 0.5},
)

print(content)

In [ ]:
with open("test.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")


message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "describe the weather in this image"
        },
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{encoded_string}"},
        },
    ],
)
response = llm.invoke([message])
print(response.content)

In [ ]:
with open("test.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

message = HumanMessage(
    content=[
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{encoded_string}"},
        },
        {
            "type": "text",
            "text": "describe the weather in this image"
        },
    ],
)
response = llm.invoke([message])
print(response.content)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "compare the two pictures provided"),
        (
            "user",
            [
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data1}"},
                },
                {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{image_data2}"},
                },
            ],
        ),
    ]
)

In [24]:
# Create a system message
from langchain_core.messages import SystemMessage, HumanMessage 

system_message = SystemMessage(content="You are a helpful assistant.")

# Create a human message (replace "Hello" with your actual query)
human_message = HumanMessage(content="Hello")

# Pass both messages to the model
response = llm.invoke([system_message, human_message])
print(response.content)

Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.


In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
        {"ai","Here\'s the German translation:\n\nIch liebe Programmieren."},
        {"human","That's great!"},
    ]
    
)
chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content="Here's the German translation:\n\nIch liebe Programmieren.", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 23, 'total_tokens': 41}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0', 'system_fingerprint': 'fp', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-693ccff6-4a54-4825-9771-2dc2d87a0606-0')